# Just to Get Hyperparameter

In [3]:
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')
import glob
import os
import pickle
import joblib
import pandas as pd
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from eli5.permutation_importance import get_score_importances
import eli5
from eli5.sklearn import PermutationImportance
from sklearn import cluster

import optuna
from optuna import Trial
from optuna.samplers import TPESampler
from optuna.visualization import plot_contour, plot_optimization_history
from optuna.visualization import plot_parallel_coordinate, plot_slice, plot_param_importances
from hyperopt.pyll.base import scope
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
import lightgbm as lgb
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import xgboost as xgb
from catboost import Pool, CatBoostClassifier
from kaggler.model import AutoLGB
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, StratifiedShuffleSplit, train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, log_loss
import random

train = pd.read_csv('data/train.csv').drop(['index', 'FLAG_MOBIL'], axis=1).fillna('NAN')
test = pd.read_csv('data/test.csv').drop(['index', 'FLAG_MOBIL'], axis=1).fillna('NAN')
sample_submission = pd.read_csv('data/sample_submission.csv')

# train데이터와 test데이터 변수를 함께 조정하기 위해 병합
merge_data = pd.concat([train, test], axis = 0)

# DAYS_BIRTH
merge_data['DAYS_BIRTH_month']=np.floor((-merge_data['DAYS_BIRTH'])/30)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/30)/12).astype(int)*12)
merge_data['DAYS_BIRTH_week']=np.floor((-merge_data['DAYS_BIRTH'])/7)-(
    (np.floor((-merge_data['DAYS_BIRTH'])/7)/4).astype(int)*4)

# DAYS_EMPLOYED
merge_data['DAYS_EMPLOYED_month']=np.floor((-merge_data['DAYS_EMPLOYED'])/30)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['DAYS_EMPLOYED_week']=np.floor((-merge_data['DAYS_EMPLOYED'])/7)-(
    (np.floor((-merge_data['DAYS_EMPLOYED'])/7)/4).astype(int)*4)

# before_EMPLOYED
merge_data['before_EMPLOYED']=merge_data['DAYS_BIRTH']-merge_data['DAYS_EMPLOYED']
merge_data['before_EMPLOYED_month']=np.floor((-merge_data['before_EMPLOYED'])/30)-(
    (np.floor((-merge_data['before_EMPLOYED'])/30)/12).astype(int)*12)
merge_data['before_EMPLOYED_week']=np.floor((-merge_data['before_EMPLOYED'])/7)-(
    (np.floor((-merge_data['before_EMPLOYED'])/7)/4).astype(int)*4)

# DAYS_BIRTH / Income
merge_data['DAYS_BIRTH_month/income_total'] = merge_data['DAYS_BIRTH_month'] / merge_data['income_total']
merge_data['DAYS_BIRTH_week/income_total'] = merge_data['DAYS_BIRTH_week'] / merge_data['income_total']

# DAYS_EMPLOYED / Income
merge_data['DAYS_EMPLOYED_month/income_total'] = merge_data['DAYS_EMPLOYED_month'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED_week/income_total'] = merge_data['DAYS_EMPLOYED_week'] / merge_data['income_total']

# before_EMPLOYED / Income
merge_data['before_EMPLOYED/income_total'] = merge_data['before_EMPLOYED'] / merge_data['income_total']
merge_data['before_EMPLOYED_month/income_total'] = merge_data['before_EMPLOYED_month'] / merge_data['income_total']
merge_data['before_EMPLOYED_week/income_total'] = merge_data['before_EMPLOYED_week'] / merge_data['income_total']

# Income / Family
merge_data['income_total/family_size'] = merge_data['income_total'] / merge_data['family_size']

merge_data['child_num/income_total'] = merge_data['child_num'] / merge_data['income_total']
merge_data['family_size/income_total'] = merge_data['family_size'] / merge_data['income_total']
merge_data['DAYS_BIRTH/income_total'] = merge_data['DAYS_BIRTH'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/income_total'] = merge_data['DAYS_EMPLOYED'] / merge_data['income_total']
merge_data['DAYS_EMPLOYED/DAYS_BIRTH'] =  merge_data['DAYS_EMPLOYED'] / merge_data['DAYS_BIRTH']

# Income skewed-data
merge_data['income_total'] = np.log1p(merge_data['income_total'])
# merge_data['log_income_total'] = np.log(merge_data['income_total'])
# merge_data['sqrt_income_total'] = np.sqrt(merge_data['income_total'])
# merge_data['boxcox_income_total'] = stats.boxcox(merge_data['income_total'])[0]

merge_data = merge_data.fillna(-999)
train = merge_data[merge_data['credit'] != -999]
test = merge_data[merge_data['credit'] == -999]
test.drop('credit', axis = 1, inplace = True)

train_cols = list(train.columns); train_cols.remove('credit'); train_cols.append('credit')
train = train[train_cols]

train_x = train.drop(['credit'], axis=1) # 데이터 나누기
train_y = train['credit']
test_x = test.copy()

enc = LabelEncoder()
for col in train_x.columns:
    if (train_x[col].dtypes=='O'):
        train_x[col] = enc.fit_transform(train_x[col])
        test_x[col] = enc.fit_transform(test_x[col])

def reduce_mem_usage(data):
    numerics = ['int8', 'int16', 'int32', 'int64', 'float32', 'float64']
    start_memory = data.memory_usage().sum() / 1024**2    
    for col in data.columns:
        col_type = data[col].dtypes
        if col_type in numerics:
            c_min = data[col].min()
            c_max = data[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    data[col] = data[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    data[col] = data[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    data[col] = data[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    data[col] = data[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    data[col] = data[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    data[col] = data[col].astype(np.float32)
                else:
                    data[col] = data[col].astype(np.float64)    
    end_memory = data.memory_usage().sum() / 1024**2
    print('Memory optimization from {:5.2f}MB to {:5.2f}MB ({:.1f}% reduction)'
          .format(start_memory, end_memory, 100 * (start_memory - end_memory) / start_memory))
    return data
train_x = reduce_mem_usage(train_x)
test_x = reduce_mem_usage(test_x)

Memory optimization from  7.67MB to  1.92MB (75.0% reduction)
Memory optimization from  2.90MB to  0.72MB (75.0% reduction)


In [18]:
def rf_objective(trial: Trial) -> float:
    params_rf = {
        "random_state": 91373,
        "criterion": "entropy",
        "n_estimators": trial.suggest_int("n_estimators", 500, 1500),
        "max_depth": trial.suggest_int("max_depth", 70, 100),
        "max_features": trial.suggest_uniform("max_features", 0.15, 0.4),
        "min_samples_split": trial.suggest_int("min_samples_split", 10, 15),
        #"min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        "max_samples": trial.suggest_uniform("max_samples", 0.8, 1),
        #"criterion": trial.suggest_categorical("criterion", ["gini", "entropy"]),
        'n_jobs': -1, 
    }
    
    # CV=10으로 튜닝
    
    seed = 91373
    kfold = StratifiedKFold(n_splits=5, random_state = seed, shuffle = True) # Cross-validation cv=5
    cv = np.zeros((train_x.shape[0], 3))

    for n, (train_idx, val_idx) in enumerate(kfold.split(train_x, train_y)):

        x_train, x_val = train_x.iloc[train_idx], train_x.iloc[val_idx]
        y_train, y_val = train_y.iloc[train_idx], train_y.iloc[val_idx]

        rfmodel = RandomForestClassifier(**params_rf)
                                                                                        
        rfmodel.fit(x_train, y_train) 
        cv[val_idx, :] = rfmodel.predict_proba(x_val)
    # print('multi_logloss:', log_loss(train_y, cv))

    
    return log_loss(train_y, cv)

In [ ]:
sampler = TPESampler(seed=42)
rf_study = optuna.create_study(study_name="rf_parameter_opt", direction="minimize", sampler=sampler)
rf_study.optimize(rf_objective, n_trials=80)

rf_best_hyperparams = rf_study.best_trial.params
rf_base_hyperparams = {'n_jobs': -1, "random_state": 91373, "criterion": 'entropy'}
rf_best_hyperparams.update(rf_base_hyperparams)
print("The best hyperparameters are:\n", rf_best_hyperparams)

In [ ]:
# 결과 나오자마자 복붙해서 카톡방에 올려서 따로 저장해두기!! (파라미터값, logg값 둘 다)

In [ ]:
plot_optimization_history(rf_study) 

In [ ]:
plot_param_importances(rf_study)

In [ ]:
plot_slice(rf_study)

In [ ]:
# 0.6888343594936606
rf_best_hyperparams = {'n_estimators': 776, 'max_depth': 95, 'max_features': 0.18938529117093866, 
                      'min_samples_split': 12, 'max_samples': 0.9089712797972337}
rf_base_hyperparams = {'random_state': 91373, 'n_jobs': -1}
rf_best_hyperparams.update(rf_base_hyperparams)

In [ ]:
# 0.6889256943243468
rf_best_hyperparams = {'n_estimators': 1014, 'max_depth': 90, 'max_features': 0.1695919256683544, 
                      'min_samples_split': 11, 'max_samples': 0.9848994211078053}
rf_base_hyperparams = {'random_state': 91373, 'n_jobs': -1}
rf_best_hyperparams.update(rf_base_hyperparams)

In [ ]:
# 위 그래프들은 지우지 말고 그대로 커밋해주기 (결과 보고 범위 다시 지정해야함)